In [1]:
import numpy as np

# from Input_Variables.read_vars import raw_data_storage, \
#                                       mfdfa_wins, mfdfa_q_list, mfdfa_rev_seg, mfdfa_pol_order, \
#                                       fpca_min_time

from Input_Variables.read_vars import raw_data_storage, \
                                      daily_stats_features_lower, daily_stats_features_upper, \
                                      ml_models_train_split, ml_models_test_split

from Data_Schema.schema import Pandas_UDF_Data_Schema
from Read_In_Data.read_data import Reading_Data
from Feature_Generation.create_binary_labels import Create_Binary_Labels
from Data_Pipeline.sklearn_pipeline import Sklearn_Pipeline
from Feature_Generation.daily_stats import Daily_Stats_Features
from Models.xgboost_model import XGBoost_Classification
# from Feature_Generation.time_series_feature_creation import TS_Feature_Creation

# General Modules

In [2]:
# PySpark UDF Schema Activation
pandas_udf_data_schema=Pandas_UDF_Data_Schema()

# Data Location
reading_data=Reading_Data(data_location=raw_data_storage)

# Create Binary y Variables
create_binary_labels=Create_Binary_Labels()

# Sklearn Pipeline
pandas_sklearn_pipeline=Sklearn_Pipeline()

# Features Daily Stats Module
daily_stats_features=Daily_Stats_Features()

# XGBoost Model Module
xgboost_classification=XGBoost_Classification()


# PySpark

### PySpark: Reading In Data

In [3]:
####### PySpark
pyspark_df=reading_data.read_in_pyspark()

### PySpark: Adding Binary Labels

In [ ]:
pyspark_df_added_binary_labels=create_binary_labels.pyspark_binary_labels(df=pyspark_df, 
                                                                          lower=daily_stats_features_lower, 
                                                                          upper=daily_stats_features_upper)

In [ ]:
pyspark_df_added_binary_labels.show(1)

### PySpark: Sklearn Pipeline in PySpark

In [ ]:
####### PySpark
pyspark_transform_schema=pandas_udf_data_schema.sklearn_pyspark_schema()
pyspark_transformations=pandas_sklearn_pipeline.pyspark_sklearn_pipeline(df=pyspark_df_added_binary_labels, 
                                                                         output_schema=pyspark_transform_schema)

In [ ]:
pyspark_transformations.show(1)

### PySpark: Features: Daily Stats

In [ ]:
####### PySpark
pyspark_features_daily_stats_schema=pandas_udf_data_schema.daily_stats_schema()

pyspark_features_daily_stats=daily_stats_features.pyspark_sklearn_pipeline(df=pyspark_transformations, 
                                                                           output_schema=pyspark_features_daily_stats_schema, 
                                                                           lower=daily_stats_features_lower, 
                                                                           upper=daily_stats_features_upper)

In [ ]:
pyspark_features_daily_stats.show(1)

### PySpark: XGBoost Model --> Trash model output --> Tester

In [ ]:
####### PySpark
pyspark_xgboost_schema=pandas_udf_data_schema.XGBoost_schema()

xgboost_classification.pyspark_xgboost(df=pyspark_features_daily_stats, 
                                       output_schema=pyspark_xgboost_schema, 
                                       train_split=ml_models_train_split, 
                                       test_split=ml_models_test_split)

# Pandas

### Pandas: Reading in Data

In [ ]:
####### Pandas
pandas_df=reading_data.read_in_pandas()

In [ ]:
pandas_df.head(1)

### Pandas: Adding Binary Labels

In [ ]:
pandas_df_added_binary_labels=create_binary_labels.pandas_binary_labels(df=pandas_df, 
                                                                        lower=daily_stats_features_lower, 
                                                                        upper=daily_stats_features_upper)

In [ ]:
pandas_df_added_binary_labels.head(1)

### Pandas: Sklearn Pipeline in Pandas

In [ ]:
####### Pandas
pandas_transformations=pandas_sklearn_pipeline.pandas_transform_features(df=pandas_df_added_binary_labels)

In [ ]:
pandas_transformations.head(1)

### Pandas: Features: Daily Stats

In [ ]:
pandas_features_daily_stats=daily_stats_features.pandas_compressDailyValues(data=pandas_transformations, 
                                                                            lower=daily_stats_features_lower, 
                                                                            upper=daily_stats_features_upper)

In [ ]:
pandas_features_daily_stats.head(1)

### Pandas: Models

In [ ]:
pat_1=pandas_features_daily_stats[pandas_features_daily_stats['PatientId']=='tHu8WPnIffml5CL+AbOBkXcbFApQnP06KdrHbjinta4=']
pat_1

### Not Ready, Do Not Run
### Need to add Summary statistics to Features
### Multifractal Detrended Fluctuation Analysis, Poincare Analysis, Functional Principal Component Analysis, Entropy

In [ ]:
ts_feature_creation=TS_Feature_Creation()

# Need to import variables from file
mfdfa=ts_feature_creation.mfdfa_extraction(data=pandas_transformations,
                                   win_sizes=mfdfa_wins,
                                   q_list=mfdfa_q_list,
                                   rev_seg=mfdfa_rev_seg,
                                   pol_order=mfdfa_pol_order)
print(mfdfa)

poincare=ts_feature_creation.poincare_extraction(data=pandas_transformations)
print(poincare)

# Fuction Works
entropy=ts_feature_creation.entropy_extraction(data=pandas_transformations)
print(entropy)

### Modeling

### TensorFlow Time Series Probability Models: [TensorFlow](https://www.tensorflow.org/probability/examples/Structural_Time_Series_Modeling_Case_Studies_Atmospheric_CO2_and_Electricity_Demand)
### PySpark XGBoost: [PySpark](https://xgboost.readthedocs.io/en/stable/tutorials/spark_estimator.html)